In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
# Carregar modelo treinado
model = load_model('mobilenetv2_emotion_finetuned_1940h_acc_0.65.keras')

In [ ]:
# Rótulos de classes (ajustar conforme necessário)
class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [ ]:
# Inicializar detector Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
# Tamanho esperado pelo modelo
img_size = 96

In [ ]:
# Inicializar webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("Não foi possível acessar a webcam.")

print("Pressione 'q' para sair.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.resize(face, (img_size, img_size))
        face_rgb = cv2.cvtColor(face_rgb, cv2.COLOR_BGR2RGB)
        face_input = np.expand_dims(face_rgb, axis=0) / 255.0

        prediction = model.predict(face_input, verbose=0)
        predicted_class = np.argmax(prediction)
        predicted_label = class_labels[predicted_class]

        # Desenhar bounding box e rótulo
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, predicted_label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Detecção de Emoções - Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()